In [1]:
import numpy as np

from default import PROJECT_HOME
%cd {PROJECT_HOME}

/Users/aflamant/Documents/courses/2024-2025/mémoire/03-code/memoire


In [87]:
from dataset import FixedPrattTrussDatasetSingleTarget
import torch
import numpy as np

ds = FixedPrattTrussDatasetSingleTarget('data/dataset/pratt_truss_bridge/single_ea/train_100000.hdf5')

In [88]:
def f(n):
    k_s = (int(ds.stiffness_matrix[n].__len__() ** .5),) * 2
    k = ds.stiffness_matrix[n].reshape(k_s)
    u = ds.nodes_displacement[n].reshape((-1, 1))
    q = ds.load[n].reshape((-1, 1))

    q_p = np.round(k @ u - q)
    q_p[[1, 17], 0] = 0.
    return q_p


r = f(0)
for i in range(1, 12):
    r += f(i)
r

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [180]:
def g(n):
    k_s = (int(ds.stiffness_matrix[n].__len__() ** .5),) * 2
    k = ds.stiffness_matrix[n].reshape(k_s).astype(np.float64)
    k /= ds[n][1]
    k[[0, 1, 17], [0, 1, 17]] = 1
    k[k.abs() < 1e-9] = 0
    return k

In [181]:
def u_from_x(x, batch_size=100, n_panels=8):
    u = torch.zeros((batch_size, 4 * n_panels))
    u[:, 2:2 * n_panels + 1] = x[:, :2 * (n_panels - 1) + 1]
    u[:, 2 * (n_panels + 1):] = x[:, 2 * (n_panels - 1) + 1:4 * n_panels - 3]
    return u.unsqueeze(-1)

def q_from_x_q(x, q, n_panels=8):
    q = q.squeeze(-1)
    q[:, np.arange(3, 2 * n_panels, 2)] = x[:, 4 * n_panels - 3: 5 * n_panels - 4]

    return q.unsqueeze(-1)

In [182]:
k_ref = g(0)
k_ref = torch.unsqueeze(k_ref, 0).to(torch.float)

In [183]:
from losses import StiffnessToLoadLoss
criterion = StiffnessToLoadLoss()

In [184]:
dl = torch.utils.data.DataLoader(ds, batch_size=100, shuffle=True)
x, y, _, u, q = dl.__iter__().__next__()
q[:, [0, 1, 17], :] = 0.

k = k_ref * y.unsqueeze(-1)
k[:, [0, 1, 17], [0, 1, 17]] = 1.

In [202]:
criterion(k*1e-6, u_from_x(x,100, 8)*1e3, q_from_x_q(x,q,8)*1e-3)

tensor(6.4561e-06)